In [1]:
# load modules to view the bonds we need to move

In [1]:
from ase.io import read, write
from ase.visualize import view
from ase import Atoms
import os
import numpy as np
import random

In [2]:
frame = read('BaseFrame.xyz', ':')
baseframe = frame[0]
baseframe.wrap(eps=1e-15)

In [3]:
view(baseframe, viewer='ngl')

In [5]:
O_atoms = []
for i in range(len(baseframe)):
    if baseframe.symbols[i] == 'O':
        O_atoms.append(i)

In [17]:
rand_O = O_atoms[random.randrange(len(O_atoms))]
print(rand_O)

160


In [20]:
# get near atoms
cell = [100, 100, 100]
positions_l = []
frame_distances = baseframe.get_all_distances()[rand_O]
for i in range(len(frame_distances)):
    if frame_distances[i] <= 3.5:
        symbol = baseframe.symbols[i]
        positions = baseframe.positions[i]
        positions_l.append(list(positions))
        print(f'{symbol} : {positions} : index {i}')
        
atoms = Atoms('OH', positions_l)

O : [ 6.84080848 13.12225804  8.08613798] : index 160
H : [ 6.42128929 12.62337035  9.76799421] : index 169


In [21]:
view(atoms, viewer='ngl')

In [22]:
Oxy = np.array([6.84080848, 13.12225804,  8.08613798])
H = np.array([6.42128929, 12.62337035,  9.76799421])

origin = np.array([0, 0, 0])

vector_d = Oxy - H
vector_d 
sum_d = 0
for i in range(vector_d.shape[0]):
    sum_d += float(vector_d[i])**2
    
mag = np.sqrt(sum_d)
print(vector_d, mag)

[ 0.41951919  0.49888769 -1.68185623] 1.803753213829875


In [24]:
b_OO = 1.803753213829875
b_O = 1.803753213829875
list_bs = []
array_BO = np.zeros([10,4])
for i in range(10):
    b_O -= 0.0803753213829875
    list_bs.append(b_O)
    vector_N = vector_d * (b_O/b_OO)
    print(f'{b_O}: {vector_N}')
    array_BO[i,0] = b_O
    array_BO[i,1] = vector_N[0]
    array_BO[i,2] = vector_N[1]
    array_BO[i,3] = vector_N[2]

array_BO
np.save('O-H_vectors.npy', array_BO)

1.7233778924468877: [ 0.4008254   0.47665723 -1.60691264]
1.6430025710639002: [ 0.3821316   0.45442678 -1.53196906]
1.5626272496809128: [ 0.36343781  0.43219632 -1.45702547]
1.4822519282979254: [ 0.34474402  0.40996586 -1.38208188]
1.401876606914938: [ 0.32605022  0.3877354  -1.3071383 ]
1.3215012855319506: [ 0.30735643  0.36550495 -1.23219471]
1.2411259641489631: [ 0.28866264  0.34327449 -1.15725112]
1.1607506427659757: [ 0.26996884  0.32104403 -1.08230754]
1.0803753213829883: [ 0.25127505  0.29881358 -1.00736395]
1.0000000000000009: [ 0.23258126  0.27658312 -0.93242036]


In [29]:
# compute the new oxygen position vector as Cartesian coordinates
H_array = np.zeros([10,3])
for i in range(array_BO.shape[0]):
    H_array[i,:] = Oxy + array_BO[i,1:]
    
H_array
np.save('H_positions.npy', H_array)

In [30]:
for i in range(H_array.shape[0]):
    new_frame = baseframe
    positions = new_frame.positions
    positions[169] = H_array[i,:]
    string = f'O-H_{round(array_BO[i,0], 5)}.xyz'
    print(positions[169])
    #write(string, new_frame)

[ 7.24163388 13.59891527  6.47922534]
[ 7.22294008 13.57668482  6.55416892]
[ 7.20424629 13.55445436  6.62911251]
[ 7.1855525 13.5322239  6.7040561]
[ 7.1668587  13.50999344  6.77899968]
[ 7.14816491 13.48776299  6.85394327]
[ 7.12947112 13.46553253  6.92888686]
[ 7.11077732 13.44330207  7.00383044]
[ 7.09208353 13.42107162  7.07877403]
[ 7.07338974 13.39884116  7.15371762]
